In [1]:
#Import statements
import json
import requests
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
from pathlib import Path
import os
import csv
import time

In [2]:
import demoji
from collections import Counter
import ast
demoji.download_codes()

... OK (Got response in 0.24 seconds)
Writing emoji data to C:\Users\Russell\.demoji/codes.json ...
... OK


In [3]:
# The stock Symbols that we will iterate through
# Apple = AAPL
# Amazon = AMZN
# Google = GOOGL
# Microsoft = MSFT
# Dell = DELL
# IBM = IBM
# Intel = INTC
# HP = HPQ
# Facebook = FB
# Cisco Systems = CSCO
# Oracle = ORCL
# HP Enterprise = HPE
# Micron Tech = MU
# DXC Tech = DXC
# Thermo Fisher Scientific = TMO
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

In [4]:
def collect_New_Twits(res):
    try:
        
        #
        df = (pd.json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment.basic','symbols'}])
        
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment.basic','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment.basic': 'tag'})
        
    except:
        df = (pd.json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment','symbols'}])
               
        #Reorders the columns and gets rid of old symbols column
        df = df[['id','body','created_at','entities.sentiment', 'symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment': 'tag'})
        

    #The following loops reformat the symbols column
    #It creates a list of dictionaries, {symbol: "symbol of company mentioned", title: "name of company"}
    dataList=[]
    
    for index in df['symbols']:
        symbolsList = []

        for diction in index:

            symbolsList.append({'symbol' : diction.get('symbol'), 'title' : diction.get('title')})
            
        dataList.append(symbolsList)
        
    df.insert(5, "newSymbols" ,dataList)
    
    df = df[['id','body','created','tag', 'newSymbols']]
    
    df = remove_emojis(df)

    #Replaces the NAN with a string "none"
    df = df.replace(np.nan, 'none', regex=True)
    
    #Replaces the string as a datetime variable
    dateFormat = "%Y-%m-%dT%H:%M:%SZ"
    df['created'] = pd.to_datetime(df['created'], format=dateFormat)
    
    df = df.sort_values(by = 'id', ascending = False)
    
    #returns the dataframe in correct format
    return df

In [5]:
##This function takes a Stocktwits dataframe and removes the emojis from the twit.
#This function then returns a dataframe with an extra column for the twit without emojis
def remove_emojis(dataframe):
    cleanSentList=[]
    
    for row in dataframe.body:
        cleanSentList.append(demoji.replace(row))
        
    dataframe.insert(5, 'cleanSents', cleanSentList)
    
    dataframe = dataframe[['id', 'cleanSents', 'created','tag', 'newSymbols']]
    
    return dataframe

In [6]:
# Get the current directory
current = os.getcwd()
print(current)

C:\Users\Russell\OneDrive - Marist College\School\Spring 2020\DATA 450 Data Capping\Repository\VM


In [7]:
for symbol in stockSymbol:
    print(symbol)
    
    symbolFolder = str(current)+"\\{}folder".format(symbol)

    #selects the file to add to
    file = symbolFolder+'\\{}_twits.csv'.format(symbol)

    historicalTwits = pd.read_csv(file)
    newestID = historicalTwits['id'].iloc[0]

    url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(symbol)
    try:
        response = requests.get(url, params = {'since' : newestID}).json()

    except:
        print("error getting request")
        try:
            response = requests.get(url, params = {'since' : newestID}).json()
        except:
            print("Second error getting request")
            continue
    if response['response']['status'] == 429:
        print("requests exceeded")
        time.sleep(600)
        continue

    if pd.json_normalize(response['messages']).empty:
        time.sleep(15)
        
        time.sleep(15)
        continue

    tempTwitsDf = collect_New_Twits(response)

    newHistoricalTwits = tempTwitsDf.append(historicalTwits)

    newHistoricalTwits = newHistoricalTwits.drop_duplicates(subset = 'id')

    newHistoricalTwits.to_csv(file, index=False)
    time.sleep(15)
    
    time.sleep(15)

AAPL
AMZN
GOOGL
MSFT
DELL
IBM
INTC
HPQ
FB
CSCO
ORCL
HPE
MU
DXC
TMO
